In [18]:
import csv
import requests
from unidecode import unidecode 
import os

In [19]:
headers = {
    'authority': 'api.sofascore.com',
    'accept': '*/*',
    'accept-language': 'pl-PL,pl;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    'if-none-match': 'W/"f810540cc3"',
    'origin': 'https://www.sofascore.com',
    'referer': 'https://www.sofascore.com/',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Opera GX";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 OPR/102.0.0.0'
}
headers['If-Modified-Since'] = 'Sun, 19 Nov 2023 00:00:00 GMT'

In [36]:
def timestamp_to_date(timestamp):
    dt_object = datetime.utcfromtimestamp(timestamp)
    formatted_date = dt_object.strftime('%Y-%m-%d')
    return formatted_date

In [37]:
def format_date(input_date):
    # Parsuj datę w formacie d.m.Y
    parsed_date = datetime.strptime(input_date, '%d.%m.%Y')
    
    # Formatuj datę z 0 przy dniu, jeśli jest jednocyfrowy
    formatted_date = parsed_date.strftime('%Y-%m-%d')
    
    return formatted_date

In [41]:
players_id_file_src = os.path.join('..', '..', 'data', 'transfermarkt', 'players_temp.csv')
players_list = []

with open(players_id_file_src, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
        
    for row in csv_reader:
        name = row['name']
        slug = row['slug'] 
        id_transfermarkt = row['id']
        date_of_birth = format_date(row['birth_date'])
        print(row['birth_date'])
        print(date_of_birth)
        player = requests.get(f'https://api.sofascore.com/api/v1/search/{slug}', headers=headers).json()
        players_details = player.get("results", [])
        correct_player_flag = False
        if len(players_details) != 0:
            for player_data in players_details:
                if player_data.get('type', "") == "player":
                    player = player_data.get("entity", {})
                    # print(slug)
                    # print(player.get("slug", ""))
                    # print(player_data)
                    # print(player.get("type", ""))
                    # print(slug)
                    # print(player.get("id", ""))
                    player_details = requests.get(f'https://api.sofascore.com/api/v1/player/{player.get("id","")}', headers=headers).json()
                    details = player_details.get("player",{})
                    date_of_birth_timestamp = details.get("dateOfBirthTimestamp", "")
                    # print(f"{timestamp_to_date(date_of_birth_timestamp)} - {date_of_birth}" )
                    if date_of_birth == timestamp_to_date(date_of_birth_timestamp):
                        player_ids = {'slug': slug,'id_transfermarkt': id_transfermarkt,'id_sofascore': player.get("id","")}
                        correct_player_flag = True
                        break
            if not correct_player_flag:
                player_ids ={'slug': slug,'id_transfermarkt': id_transfermarkt,'id_sofascore': None}
        else: 
            player_ids ={'slug': slug,'id_transfermarkt': id_transfermarkt,'id_sofascore': None}
        players_list.append(player_ids)
            

28.06.2003
2003-06-28
5.07.2000
2000-07-05
23.07.2000
2000-07-23
20.03.2001
2001-03-20
10.03.2002
2002-03-10
3.05.2000
2000-05-03
27.03.2000
2000-03-27
27.03.2001
2001-03-27
16.12.2002
2002-12-16
26.01.2001
2001-01-26
24.06.1999
1999-06-24
6.06.1999
1999-06-06


In [22]:
output_filename = os.path.join('..', '..', 'data', 'players', 'players_ids_temp.csv')
with open(output_filename, 'w', newline='') as output_csv_file:
    fieldnames = ['slug', 'id_transfermarkt', 'id_sofascore']
    csv_writer = csv.DictWriter(output_csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    csv_writer.writerows(players_list)    

In [8]:
response = requests.get(f'https://api.sofascore.com/api/v1/player/927724', headers=headers).json()
response

{'player': {'name': 'Piotr Świątko',
  'slug': 'piotr-swiatko',
  'shortName': 'P. Świątko',
  'team': {'name': 'No team',
   'slug': 'no-team',
   'shortName': 'No team',
   'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
   'userCount': 13,
   'nameCode': 'NTE',
   'disabled': True,
   'national': False,
   'type': 0,
   'id': 241802,
   'country': {},
   'teamColors': {'primary': '#d0d0d0',
    'secondary': '#000000',
    'text': '#000000'}},
  'position': 'D',
  'height': 182,
  'preferredFoot': 'Right',
  'userCount': 3,
  'id': 927724,
  'country': {'alpha2': 'PL', 'name': 'Poland'},
  'dateOfBirthTimestamp': 927936000,
  'proposedMarketValue': 46000,
  'proposedMarketValueRaw': {'value': 46000, 'currency': 'EUR'}}}

In [6]:
dateBirth = response.get("player", {}).get("dateOfBirthTimestamp","")

In [7]:
from datetime import datetime
dt_object = datetime.utcfromtimestamp(dateBirth)
formatted_date = dt_object.strftime('%Y-%m-%d %H:%M:%S')
formatted_date

'1999-05-29 00:00:00'

In [13]:
player = requests.get(f'https://api.sofascore.com/api/v1/search/piotr-swiatko', headers=headers).json()
player

{'results': [{'type': 'player',
   'entity': {'name': 'Piotr Świątko',
    'slug': 'piotr-swiatko',
    'shortName': 'P. Świątko',
    'team': {'name': 'No team',
     'slug': 'no-team',
     'shortName': 'No team',
     'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
     'userCount': 13,
     'nameCode': 'NTE',
     'disabled': True,
     'national': False,
     'type': 0,
     'id': 241802,
     'country': {},
     'subTeams': [],
     'teamColors': {'primary': '#d0d0d0',
      'secondary': '#000000',
      'text': '#000000'}},
    'position': 'D',
    'userCount': 3,
    'id': 927724,
    'country': {'alpha2': 'PL', 'name': 'Poland'}},
   'score': 2398.8665}]}